In [1]:
import argparse
import cv2
import glob
import os
from basicsr.archs.rrdbnet_arch import RRDBNet
from tqdm import tqdm
from realesrgan import RealESRGANer
import numpy as np

In [2]:
netscale = 4
tile = 0
tile_pad = 10
pre_pad = 0
half=""
outscale = 4 # Во сколько раз увеличить разрешение

In [3]:
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=netscale)

In [4]:
model_path = "./model/RealESRGAN_x4plus_anime_6B.pth"

In [5]:
upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path,
        model=model,
        tile=tile,
        tile_pad=tile_pad,
        pre_pad=pre_pad,
        half=half)

In [11]:
import skvideo.io

In [12]:
videodata = skvideo.io.vread("akira.mp4")

In [13]:
frame_width = int(videodata.shape[2])
frame_height = int(videodata.shape[1])
print("Разрешение видео:", frame_width, "x", frame_height)

Разрешение видео: 640 x 360


In [81]:
metadata = skvideo.io.ffprobe("akira.mp4")
fps = float(metadata["video"]['@r_frame_rate'].split('/')[0])
print("Частота кадров:", fps)

Частота кадров: 25.0


In [82]:
frame_count = videodata.shape[0]
print('Количестов кадров:', frame_count)  # float `frame_count`
print('Длительность видео (сек):', frame_count/fps)  # float `frame_count`

Количестов кадров: 3123
Длительность видео (сек): 124.92


In [83]:
output, _ = upsampler.enhance(videodata[0], outscale=outscale)
height = output.shape[0]
width = output.shape[1]
print("Выходное разрешение видео:", width, "x", height)

Выходное разрешение видео: 2560 x 1440


In [35]:
crf = 17
writer = skvideo.io.FFmpegWriter('akira_output_3.avi',
            inputdict={'-r': str(fps), '-s':'{}x{}'.format(width,height)},
            outputdict={'-r': str(fps), '-c:v': 'libx264', '-crf': str(crf), '-preset': 'veryslow', '-pix_fmt': 'yuv444p'}
)

25.0

In [69]:
for frame in tqdm(videodata):
    output, _ = upsampler.enhance(frame, outscale=outscale)
    writer.writeFrame(output)
writer.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 3123/3123 [24:52<00:00,  2.09it/s]


In [ ]:
# Так как видео сохраняется без звука, достанем звук из оригинального видео
!ffmpeg -i akira.mp4 akira.mp3

In [ ]:
# И добавим его в наш видос
!ffmpeg -i akira_output_3.avi -i akira.mp3 -c copy -map 0:v:0 -map 1:a:0 akira_output_audio.avi